In [ ]:
# Passo 1 Importar bibliotecas
import pandas as pd  # Para dataframe
import geopandas as gpd  # Para dados geo
import folium  # Para criar os mapas
from geopy.geocoders import Nominatim  # Para pegar endereços (latitude Longitude)
from shapely.geometry import Point  # Para as geometrias

# Passo 2: Carregar os dados

df_mun_func = pd.read_csv('E:/GitHub/UFU_MAPPERS/microdados/dados/microdados_uberlandia_em_funcionamento_reduzido.csv', delimiter = ';',
                           encoding = 'iso-8859-1', low_memory=False)

In [2]:
# Passo 3: Criar uma função de geocodificação com o Geopy
def geocode_address(address):
    geolocator = Nominatim(user_agent="uberlandia_locator")  # Criar o geocoder
    location = geolocator.geocode(address)  # Solicitar coordenadas do endereço
    if location:
        return Point(location.longitude, location.latitude)  # Retornar como ponto
    else:
        return None  # Retornar NONE se o endereço nao for localizado


    NU_ANO_CENSO NO_REGIAO  CO_REGIAO         NO_UF SG_UF  CO_UF NO_MUNICIPIO  \
0           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
1           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
2           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
3           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
4           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
5           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
6           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
7           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
8           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
9           2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
10          2022   Sudeste          3  Minas Gerais    MG     31   Uberlândia   
11          2022   Sudeste  

In [ ]:
# Passo 4: Criar um endereço novo completo
df_mun_func['full_address'] = 'Brasil' + ', ' + 'MG' + ', ' + df_mun_func['NO_MUNICIPIO'] + ', ' + df_mun_func['DS_ENDERECO'] + ', ' + df_mun_func['NU_ENDERECO']



In [ ]:
# Passo 5: Aplicar a função de geocodificação
df_mun_func['geometry'] = df_mun_func['full_address'].apply(geocode_address)

In [ ]:
# Passo 6: Criar o GeoDataFrame
gdf = gpd.GeoDataFrame(df_mun_func, geometry='geometry')


In [ ]:
# Passo 7: Criar um mapa com as localizações desejadas
#Substituir as coordenadas iniciais com as coordenadas desejadas
# No caso de Uberlandia, as coordenadas são: -18.9186, -48.2772
map_center = (-18.9186, -48.2772)
m = folium.Map(location=map_center, zoom_start=13)

In [ ]:
# Passo 8
# Fazer loop por cada linha do gdf e gerar a marcação da edificação
for idx, row in gdf.iterrows():
    if not pd.isnull(row['geometry']):
        folium.Marker(location=[row.geometry.y, row.geometry.x], popup=row['NO_ENTIDADE']).add_to(m)

In [ ]:
# Passo 9: Salvar o mapa em html
m.save('E:/GitHub/UFU_MAPPERS/microdados/dados/building_locations_map_2.html')